In [58]:
import jax.numpy as jnp
from jax import grad, jit, random, tree_util, vmap
from jax.nn import relu
from jax.scipy.special import logsumexp
import tensorflow as tf
import tensorflow_datasets as tfds

In [59]:
# Load MNIST from TensorFlow Datasets
data_dir = '/tmp/tfds' # data_dir = './data/tfds'
mnist_data, info = tfds.load(name="mnist", batch_size=-1, data_dir=data_dir, with_info=True)

In [60]:
def normalise(x, x_max=255.0):
    return x / x_max

def convert_to_jax(data_np, data_type):
    if data_type == "image":
        data_jax = normalise(jnp.array(data_np, dtype=jnp.float32))
    elif data_type == "label":
        data_jax = jnp.array(data_np)
    else:
        raise ValueError("not image or label")
    return data_jax

def flatten_image_for_mlp(data_jax):
    n_batch, n_pixels_vertical, n_pixels_horizontal, n_channels = data_jax.shape
    data_flattened = data_jax.reshape(n_batch, -1)
    return data_flattened

def prepare_data(data_dict):
    data_jax = {}
    for data_type, data_tf in data_dict.items():
        data_numpy = data_tf.numpy()
        data_jax[data_type] = convert_to_jax(data_numpy, data_type)
        if data_type == "image":
            data_jax[data_type] = flatten_image_for_mlp(data_jax[data_type])
    return data_jax

In [61]:
dataset_tf = "train"
all_data_tf = mnist_data[dataset_tf]
all_data_jax = prepare_data(all_data_tf)

In [62]:
images = all_data_jax["image"]
labels = all_data_jax["label"]

In [63]:
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

Images shape: (60000, 784)
Labels shape: (60000,)


In [64]:
def calculate_preactivations(x, W, b):
    """
    x is a row vector (single sample) - shape: (784,)
    W is the weights matrix - shape: (784, 128)
    b is the bias vector - shape: (128,)
    Returns: row vector - shape: (128,)
    """
    return x @ W + b

In [65]:
def forward_pass(x, params_list):
    """Forward pass through all layers: inputs and outputs are vectors"""
    for layer_number, params in enumerate(params_list):
        W, b = params["W"], params["b"]
        x = calculate_preactivations(x, W, b)
        if layer_number != (len(params_list) - 1):
            x = relu(x)
    return x

In [66]:
def calculate_loss(predictions_logits, observed_label):
    log_probs = predictions_logits - logsumexp(predictions_logits)
    return -log_probs[observed_label] 

In [67]:
forward_pass_batch = vmap(forward_pass, in_axes=(0, None))
calculate_loss_batch = vmap(calculate_loss, in_axes=(0, 0))

In [ ]:
def initialise_params(dims, key, scale=0.
1):
    return scale * random.normal(key, dims)

def random_layer_params(m, n, key):
    """Initialize weights and biases for one layer"""
    w_key, b_key = random.split(key)
    return {
        "W": initialise_params((m, n), w_key),
        "b": initialise_params((n,), b_key)
    }

def init_network_params(sizes, key):
    """Initialize all layers for a fully-connected neural network"""
    params_list = []
    keys = random.split(key, len(sizes) - 1)
    
    for i in range(len(sizes) - 1):
        input_size = sizes[i]
        output_size = sizes[i + 1]
        layer_key = keys[i]
        layer_params = random_layer_params(input_size, output_size, layer_key)
        params_list.append(layer_params)
    
    return params_list

In [76]:
layer_sizes = [784, 128, 10]
params = init_network_params(layer_sizes, random.key(0))

trial_set_size = 100
test_images = images[:trial_set_size]
test_labels = labels[:trial_set_size]

logits = forward_pass_batch(test_images, params)
predictions = jnp.argmax(logits, axis=1)

# Display results
print("True labels:    ", test_labels)
print("Predictions:    ", predictions)
print("Match:          ", predictions == test_labels)

True labels:     [4 1 0 7 8 1 2 7 1 6 6 4 7 7 3 3 7 9 9 1 0 6 6 9 9 4 8 9 4 7 3 3 0 9 4 9 0
 6 8 4 7 2 6 0 3 1 1 7 2 4 4 6 5 1 9 3 2 4 3 4 4 7 5 8 1 1 4 1 5 3 5 8 4 1
 1 4 5 3 2 4 1 4 8 1 2 1 9 0 7 6 7 4 4 9 7 5 6 8 4 6]
Predictions:     [7 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 5 1 1 1 7 1 7 7 7 1 1 1 1 1 5 7 1
 1 7 5 1 1 7 1 7 1 7 1 1 1 7 7 1 1 5 1 1 5 1 1 5 1 1 1 1 1 7 1 0 1 1 7 7 1
 1 5 7 1 1 1 1 7 1 7 1 1 7 1 1 7 1 1 5 1 1 1 1 1 7 1]
Match:           [False  True False False False  True False False  True False False False
 False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False  True False False
 False False False False False  True False False False False False False
 False False False False  True  True False  True False False False False
 False  True  True False False False False False  True False False False
 False  True False False False

### TODO
- vmap instead of having batch as part of the input matrix dimension
- initialisation of parameters
- backprop

### Data details
- n cols x n cells x n colour channels
- logits output (10 classes)

### Tutorials
- https://docs.jax.dev/en/latest/notebooks/neural_network_with_tfds_data.html
- https://flax.readthedocs.io/en/latest/mnist_tutorial.html

---

# 🚀 The Minimal JAX Deep-Learning Stack (High-Level Only)

## **1. JAX — the raw math + autodiff + transforms**

**You’re using it for:**

* `jax.numpy` (matrix mults, activations, reshaping)
* `jax.grad` (compute gradients)
* `jit`, `vmap` (speed + batching)
* RNGs (`jax.random`)

**Job description:**
**“I handle all the math and turn your code into fast GPU/TPU kernels.”**

👉 Absolutely essential. Everything else is optional helpers layered on top.

---

## **2. Flax — model architecture (layers & parameters)**

**You’re currently doing:**

* Manually creating weight matrices (`W1`, `b1`, etc.)
* Writing forward functions that manipulate those arrays
* Juggling parameter PyTrees yourself

**Flax does:**

* Provides `Dense`, `Conv`, `Dropout`, etc.
* Organises parameters into a clean structure
* Lets you write models like:

```python
class MLP(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(128)(x)
        x = nn.relu(x)
        x = nn.Dense(10)(x)
        return x
```

**Job description:**
**“I manage your layers, parameters, and model structure so you don’t.”**

Two APIs:

* **Linen** — the classic one in most tutorials
* **NNX** — newer, simpler one for future projects
  (but they solve the same problem)

👉 Not required to *learn JAX*, but required to build *bigger, nicer* models without going insane.

---

## **3. Optax — gradient updates & optimizers**

**You’re currently doing:**

* Writing your own SGD / Adam
* Updating parameters manually: `params = params - lr * grads`

**Optax does:**

* Provides Adam, AdamW, SGD, RMSProp, schedules, clipping
* Handles optimizer state automatically

Usage looks like:

```python
opt = optax.adam(learning_rate)
opt_state = opt.init(params)
updates, opt_state = opt.update(grads, opt_state)
params = optax.apply_updates(params, updates)
```

**Job description:**
**“I update your model parameters in a clean, reusable way.”**

👉 Makes optimizers easy. Almost everyone uses Optax with JAX/Flax.

---

## **4. Orbax — saving & loading (checkpointing)**

**Right now:**

* You might be doing nothing or just `pickle`ing parameters.

**Orbax does:**

* Saves model + optimizer state robustly
* Restores checkpoints cleanly
* Supports multi-device training

**Job description:**
**“I store your model safely so you can resume training.”**

👉 Only really needed for real training workflows, not toy MNIST scripts.

---

## **5. TFDS — dataset loading (optional)**

**Right now:**

* You might be writing your own MNIST loader.

---

# 🎯 The whole picture in one sentence

> **JAX** does the math and autodiff.
> **Flax** builds the model.
> **Optax** updates the model.
> **Orbax** saves the model.
